In [1]:
%load_ext autoreload
%autoreload 2

from common import *
from config import *
import utils
from models.model import*

module://ipykernel.pylab.backend_inline
@common.py:  
	set random seed
		SEED=35202
	set cuda environment
		torch.__version__              = 1.0.0
		torch.version.cuda             = 9.0.176
		torch.backends.cudnn.version() = 7401
		torch.cuda.device_count()      = 1



In [2]:
config = DefaultConfigs()


# 1. set random seed
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpus
try:
    print('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =', os.environ['CUDA_VISIBLE_DEVICES'])
    NUM_CUDA_DEVICES = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
except Exception:
    print('\t\tos[\'CUDA_VISIBLE_DEVICES\']     =', 'None')
    NUM_CUDA_DEVICES = 1
warnings.filterwarnings('ignore')

if not os.path.exists('../results'):
    os.mkdir('../results')

if not os.path.exists(config.logs):
    os.mkdir(config.logs)

log = utils.Logger()
log.open('{0}{1}_log_train.txt'.format(config.logs, config.model_name),mode="a")

log.write("\n-------------------- [START %s] %s\n\n" % (datetime.now().strftime('%Y-%m-%d %H:%M:%S'), '-' * 51))
log.write('                          |------ Train ------|------ Valid ------|----Best Results---|------------|\n')
log.write('mode    iter   epoch    lr|  loss    f1_macro |  loss    f1_macro |  loss    f1_macro | time       |\n')
log.write('----------------------------------------------------------------------------------------------------\n')

		os['CUDA_VISIBLE_DEVICES']     = 0

-------------------- [START 2019-01-18 04:13:20] ---------------------------------------------------

                          |------ Train ------|------ Valid ------|----Best Results---|------------|
mode    iter   epoch    lr|  loss    f1_macro |  loss    f1_macro |  loss    f1_macro | time       |
----------------------------------------------------------------------------------------------------


In [3]:
 # 4.1 mkdirs
if not os.path.exists(config.submit):
    os.makedirs(config.submit)
if not os.path.exists(config.weights + config.model_name + os.sep + 'fold_'+str(config.fold)):
    os.makedirs(config.weights + config.model_name + os.sep + 'fold_'+ str(config.fold))
if not os.path.exists(config.best_models):
    os.mkdir(config.best_models)
if not os.path.exists(config.logs):
    os.mkdir(config.logs)

tqdm.pandas()

start_time = time.time()
train_X, test_X, train_y, word_index = utils.load_and_prec(config)
# embedding_matrix_1 = load_glove(word_index)
# embedding_matrix_2 = load_para(word_index)

total_time = (time.time() - start_time) / 60
print("Took {:.2f} minutes".format(total_time))

  3%|▎         | 37785/1306122 [00:00<00:03, 377847.66it/s]

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


100%|██████████| 56370/56370 [00:00<00:00, 69142.46it/s]


Tokenizing.......
Tokenizing Done!
Took 2.80 minutes


In [4]:
# -------------------------------------------------------
# training
# -------------------------------------------------------
train_preds = np.zeros((len(train_X)))
test_preds = np.zeros((len(test_X)))

x_test_cuda = torch.tensor(test_X, dtype=torch.long).cuda()
test = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test, batch_size=config.batch_size, shuffle=False)

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(train_X, train_y))

In [ ]:
sigmoid = nn.Sigmoid()
for i, (train_idx, valid_idx) in enumerate(splits):
    x_train_fold = torch.tensor(train_X[train_idx], dtype=torch.long).cuda()
    y_train_fold = torch.tensor(train_y[train_idx, np.newaxis], dtype=torch.float32).cuda()
    x_val_fold = torch.tensor(train_X[valid_idx], dtype=torch.long).cuda()
    y_val_fold = torch.tensor(train_y[valid_idx, np.newaxis], dtype=torch.float32).cuda()

    model = Baseline_Bidir_LSTM_GRU(config, word_index)
    model.cuda()

    loss_fn = torch.nn.BCEWithLogitsLoss(reduction="sum")
    optimizer = torch.optim.Adam(model.parameters())

    train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)

    train_loader = torch.utils.data.DataLoader(train, batch_size=config.batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=config.batch_size, shuffle=False)

    print(f'Fold {i + 1}')

    for epoch in range(config.epochs):
        start_time = time.time()

        model.train()
        avg_loss = 0.
        for x_batch, y_batch in tqdm(train_loader, disable=True):
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)

        model.eval()
        valid_preds_fold = np.zeros((x_val_fold.size(0)))
        test_preds_fold = np.zeros(len(test_X))
        avg_val_loss = 0.
        
        # validate
        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred = model(x_batch).detach()
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            valid_preds_fold[i * config.batch_size:(i + 1) * config.batch_size] = sigmoid(y_pred).cpu().numpy()[:, 0]

        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
            epoch + 1, config.epochs, avg_loss, avg_val_loss, elapsed_time))

    # test    
    for i, (x_batch,) in enumerate(test_loader):
        y_pred = model(x_batch).detach()

        test_preds_fold[i * config.batch_size:(i + 1) * config.batch_size] =  sigmoid(y_pred).cpu().numpy()[:, 0]

    train_preds[valid_idx] = valid_preds_fold
    test_preds += test_preds_fold / len(splits)

Start loading embedding....................


In [25]:
m = nn.Sigmoid()
m(y_pred).cpu().numpy()[:,0].shape

(1536,)

In [18]:
torch.nn.Sigmoid([[0]])

TypeError: __init__() takes 1 positional argument but 2 were given